In [ ]:
import cv2
import json
import glob
import os

images_dir = './flight_1 martina'
labels_dir = './flight_1 martina'

image_paths = sorted(glob.glob(os.path.join(images_dir, '*.JPG')))

for img_path in image_paths:
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(labels_dir, base_name + '.json')
    
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    if not os.path.exists(label_path):
        continue
    with open(label_path, 'r') as f:
        data = json.load(f)
    
    for shape in data.get('shapes', []):
        points = shape.get('points', [])
        if len(points) == 2:
            pt1 = (int(points[0][0]), int(points[0][1]))
            pt2 = (int(points[1][0]), int(points[1][1]))
            cv2.rectangle(img, pt1, pt2, (0, 255, 0), 2)
    
    os.makedirs('./images2', exist_ok=True)
    output_path = os.path.join('./images2', os.path.basename(img_path))
    cv2.imwrite(output_path, img)

cv2.destroyAllWindows()

In [ ]:
labels_dir = './flight_1_martina_yolo8_labels'
images_dir = './flight_1_martina'
image_paths = sorted(glob.glob(os.path.join(images_dir, '*.JPG')))

for img_path in image_paths:
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(labels_dir, base_name + '.txt')

    print(f"Processing {img_path} with label {label_path}")
    
    img = cv2.imread(img_path)
    if img is None:
        continue

    if not os.path.exists(label_path):
        continue

    # Read image dimensions
    h, w = img.shape[:2]

    

    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            _, x_center, y_center, width, height = map(float, parts)
            x_center *= w
            y_center *= h
            width *= w
            height *= h
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    os.makedirs('./images3', exist_ok=True)
    output_path = os.path.join('./images3', os.path.basename(img_path))
    cv2.imwrite(output_path, img)

In [ ]:
import shutil
import os
import json
import glob
import cv2

input_json_dir = './flight_1 martina'
output_labels_dir = './flight_1_martina_yolo8_labels'
image_dir = './flight_1 martina'

os.makedirs(output_labels_dir, exist_ok=True)

for json_file in glob.glob(os.path.join(input_json_dir, '*.json')):
    with open(json_file, 'r') as f:
        data = json.load(f)

    base_name = os.path.splitext(os.path.basename(json_file))[0]
    image_path = os.path.join('./flight_1 martina', base_name + '.JPG')
    
    if not os.path.exists(image_path):
        continue

    img = cv2.imread(image_path)
    if img is None:
        continue

    h, w = img.shape[:2]

    yolo_lines = []
    for shape in data.get('shapes', []):
        points = shape.get('points', [])
        if len(points) == 2:
            x1, y1 = points[0]
            x2, y2 = points[1]
            x_min, x_max = min(x1, x2), max(x1, x2)
            y_min, y_max = min(y1, y2), max(y1, y2)
            x_center = (x_min + x_max) / 2 / w
            y_center = (y_min + y_max) / 2 / h
            width = (x_max - x_min) / w
            height = (y_max - y_min) / h
            label = shape.get('label', '')
            if label == 'target':
                class_id = 0
            elif label == 'boat':
                class_id = 1
            elif label == 'drifter':
                class_id = 2
                
            yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}")

    base_name = os.path.splitext(os.path.basename(json_file))[0]
    label_file = os.path.join(output_labels_dir, base_name + '.txt')
    with open(label_file, 'w') as f:
        f.write('\n'.join(yolo_lines))

In [ ]:
import glob
import json
import os

json_dir = 'zvone'
label_names = set()

for json_path in glob.glob(os.path.join(json_dir, '*.json')):
    with open(json_path, 'r') as f:
        data = json.load(f)
        for shape in data.get('shapes', []):
            label = shape.get('label')
            if label is not None:
                label_names.add(label)

print("Unique label names:", label_names)

In [8]:
import glob
import os

input_json_dir_stella = './flight_1 stella'

class_examples = {0: None, 1: None, 2: None}

for json_file in glob.glob(os.path.join(input_json_dir_stella, '*.txt')):
    with open(json_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id = int(parts[0])
                if class_id in class_examples and class_examples[class_id] is None:
                    base_name = os.path.splitext(os.path.basename(json_file))[0]
                    class_examples[class_id] = base_name + '.JPG'
    if all(v is not None for v in class_examples.values()):
        break

print("Example images for each class:")
for class_id, img_name in class_examples.items():
    print(f"Class {class_id}: {img_name}")

Example images for each class:
Class 0: DJI_0005.JPG
Class 1: DJI_0001.JPG
Class 2: DJI_0032.JPG
